In [1]:
import pandas as pd
import numpy as np

In [2]:
benchmark_weights = pd.read_excel(
    io="../data/raw/Technical Test - Portfolio Attribution.xlsm",
    sheet_name="Benchmark Weights",
    header=[0, 1],
    index_col=0)

saa_weights = pd.read_excel(
    io="../data/raw/Technical Test - Portfolio Attribution.xlsm",
    sheet_name="SAA Weights",
    header=[0, 1],
    index_col=0)

manager_weights = pd.read_excel(
    io="../data/raw/Technical Test - Portfolio Attribution.xlsm",
    sheet_name="Manager Weights",
    header=[0, 1, 2],
    index_col=0)

returns = pd.read_csv("../data/interim/returns.csv", header=[0, 1], index_col=[0])

In [147]:
class ReturnSeriesCalculator:

    def __init__(self, benchmark_weights, saa_weights, manager_weights, returns):

        returns.index = pd.to_datetime(returns.index)

        # in general here dropping multi-indexes and renaming columns
        # for consistency and ease of use/readablity
        self.index_returns = returns["Index Total Returns (CAD)"]
        self.fund_returns = returns["Fund Total Returns (CAD)"]

        self.benchmark_weights = benchmark_weights.droplevel(0, axis=1)
        self.benchmark_weights = self.benchmark_weights.rename(
            columns=dict(zip(self.benchmark_weights.columns,
                             self.index_returns.columns[0:5])))

        self.saa_weights = saa_weights.droplevel(0, axis=1)
        self.saa_weights = self.saa_weights.rename(
            columns=dict(zip(self.saa_weights.columns,
                             self.index_returns.columns)))

        manager_weights = manager_weights.droplevel(0, axis=1)
        man_sub_cats = list(
            manager_weights.columns.get_level_values(0).unique())

        # switching order to make sure US and Canadian equity funds
        # get assigned correctly
        man_sub_cats[0], man_sub_cats[1] = man_sub_cats[1], man_sub_cats[0]

        # slightly heavy looking code here but we are creating a dictionary
        # in which the keys are the return categories of the index and the
        # values are dataframes of the fund allocations for that particular
        # asset type. Each dataframe's rows in this dict add up to 1. It is
        # convenient to have these dfs in this format for fund_allocation_gen()
        self.manager_weights = {
            return_category: fund_df for (return_category, fund_df) in zip(
                self.index_returns.columns,
                [manager_weights[man_sub_cats[i]] for i in
                 range(0, len(man_sub_cats))])
        }

    def return_gen(self, return_df, allocation_df):
        np.testing.assert_array_equal(return_df.columns, allocation_df.columns)

        for date, returns in return_df.iterrows():

            # the searchsorted part gets the index location of the most recent
            # allocation decision given our date being passed in the for loop
            # we use side=left since allocations are end of day
            # we have to use the len() - [::-1].search trick because the
            # dateindex of the data is all in descending order
            allocation = allocation_df.iloc[
                len(allocation_df.index) -
                allocation_df.index[::-1].searchsorted(
                    date, side='left')
            ]

            assert np.isclose(allocation.sum(), 1), f"{allocation} sum is not 1"
            # the allocation decision date should
            # always be behind the date it is used given EOD rebalance
            assert date > allocation.name

            yield pd.DataFrame(returns).T * allocation

    def get_benchmark_returns(self):

        self.benchmark_returns = pd.concat(

            [df for df in self.return_gen(
                self.index_returns.loc[:, self.benchmark_weights.columns],
                self.benchmark_weights)]

        ).sum(axis=1)

        assert np.isclose(
            (self.index_returns.iloc[0, 0:5] * self.benchmark_weights.iloc[0]).sum(),
            self.benchmark_returns[0])

    def get_saa_returns(self):

        self.saa_returns = pd.concat(

            [df for df in self.return_gen(
                self.index_returns,
                self.saa_weights)]

        ).sum(axis=1)

        # make sure manual calculations checks out for first and last row
        assert np.isclose(
            (self.index_returns.iloc[-1] * self.saa_weights.iloc[-1]).sum(),
            self.saa_returns[-1])

        assert np.isclose(
            (self.index_returns.iloc[0] * self.saa_weights.iloc[0]).sum(),
            self.saa_returns[0])

    def fund_allocation_gen(self, asset_fund_df, saa_weights, asset_name):
        # generates allocations per fund taking into account the SAA
        saa_asset_index = list(saa_weights.columns).index(asset_name)

        for date, within_asset_fund_allocation in asset_fund_df.iterrows():

            asset_allocation = saa_weights.iloc[

                # in this case we use right side because both
                # of these weights are being used at EOD
                len(saa_weights.index) -
                saa_weights.index[::-1].searchsorted(
                    date, side='right'),

                saa_asset_index]

            yield pd.DataFrame(within_asset_fund_allocation).T * asset_allocation

    def fund_return_gen(self, fund_returns_df, fund_weights):
        np.testing.assert_array_equal(fund_returns_df.columns, fund_weights.columns)

        for date, fund_returns in fund_returns_df.iterrows():

            fund_allocation = fund_weights.iloc[

                len(fund_weights.index) -
                fund_weights.index[::-1].searchsorted(
                    date, side='left')]

            assert date > fund_allocation.name

            yield pd.DataFrame(fund_returns).T * fund_allocation

    def get_manager_returns(self):

        temp_fund_asset_dict = {}

        for asset in self.manager_weights.keys():

            temp_fund_asset_dict[asset] = pd.concat(

                [df for df in self.fund_allocation_gen(
                    asset_fund_df=self.manager_weights[asset],
                    saa_weights=self.saa_weights,
                    asset_name=asset
                )]
            )

            # make sure manual calc checks out for first and last rows
            np.testing.assert_array_almost_equal(
                self.manager_weights[asset].iloc[0] * self.saa_weights.iloc[0][asset],
                temp_fund_asset_dict[asset].iloc[0])

            np.testing.assert_array_almost_equal(
                self.manager_weights[asset].iloc[-1] * self.saa_weights.iloc[-1][asset],
                temp_fund_asset_dict[asset].iloc[-1])

        temp_fund_allocation_list = [
            temp_fund_asset_dict[asset] for asset in temp_fund_asset_dict.keys()]

        # switch back order of colnames to match fund returns
        temp_fund_allocation_list[0], temp_fund_allocation_list[1] = temp_fund_allocation_list[1], temp_fund_allocation_list[0]

        # save this as attribute since weights will be useful for drift analysis
        self.fund_saa_allocation = pd.concat(temp_fund_allocation_list, axis=1)

        # weights should sum to 1 across all assets now instead of within asset
        np.testing.assert_array_almost_equal(
            self.fund_saa_allocation.sum(axis=1),
            np.ones(len(self.fund_saa_allocation)))

        self.manager_returns = pd.concat(

            [df for df in
             self.fund_return_gen(fund_returns_df=self.fund_returns,
                                  fund_weights=self.fund_saa_allocation)],
            axis=0

        ).sum(axis=1)

    def create_return_comparison_df(self):

        self.return_comparison_df = pd.DataFrame(
            {"Benchmark Returns": self.benchmark_returns,
             "SAA Returns": self.saa_returns,
             "Manager Returns": self.manager_returns,
             "Manager Returns With Drift": self.manager_returns_with_drift})

    def write_csv(self, path):
        self.return_comparison_df.to_csv(path)

    def get_manager_returns_with_drift(self):
        # we will simulate each day to get these returns

        # utility functions for this process
        def apply_one_day_return(start_value, daily_return):
            return start_value.set_index(daily_return.index) * daily_return

        def daily_fund_return_gen(fund_return_df, fund_saa_allocation):
            # yields daily returns in ascending date order
            # adding 1 makes returns ready for multiplication
            for date, fund_return in (fund_return_df[::-1] + 1).iterrows():
                yield(pd.DataFrame(fund_return).T)

        def rebalance(current_value, fund_weights):
            total_value = current_value.sum(axis=1)[0]
            return fund_weights * total_value

        fund_value = pd.DataFrame(self.fund_saa_allocation.iloc[-1]).T
        start_value = fund_value

        for daily_return in daily_fund_return_gen(self.fund_returns, self.fund_saa_allocation):

            fund_value = pd.concat(
                [apply_one_day_return(start_value, daily_return),
                 fund_value])

            start_value = pd.DataFrame(fund_value.iloc[0]).T

            if start_value.index[0] in self.fund_saa_allocation.index:

                start_value = rebalance(
                    start_value,
                    pd.DataFrame(self.fund_saa_allocation.loc[start_value.index[0]]).T)

        portfolio_value = fund_value.sum(axis=1)

        self.manager_returns_with_drift = portfolio_value[::-1].pct_change()[::-1].dropna()

    def get_all_returns(self):
        self.get_benchmark_returns()
        self.get_saa_returns()
        self.get_manager_returns()
        self.get_manager_returns_with_drift()
        self.create_return_comparison_df()

In [196]:
class IndexCalculator(ReturnSeriesCalculator):

    def get_cumulative_return(self):
        self.cumulative_return_df = (
            self.return_comparison_df + 1)[::-1].cumprod(axis=0)[::-1] - 1

    def get_indexed_df(self, date="2020-03-06"):
        assert (pd.to_datetime(date) in
                self.cumulative_return_df.index), "date should be str in format YYYY-MM-DD and be in index"
        date = pd.to_datetime(date)

        one_plus_cumulative_return = self.cumulative_return_df + 1
        index_row = one_plus_cumulative_return.loc[date]

        self.indexed_df = one_plus_cumulative_return.div(index_row, axis=1) * 100

In [192]:
test_ic = IndexCalculator(benchmark_weights, saa_weights, manager_weights, returns)

test_ic.get_all_returns()

In [193]:
test_ic.get_cumulative_return()

In [ ]:
test_ic.get_indexed_df(date="2022-10-26")

In [ ]:
test_ic.indexed_df

,Benchmark Returns,SAA Returns,Manager Returns,Manager Returns With Drift
2022-10-31,100.672899,100.669012,100.389071,100.385127
2022-10-28,100.844450,100.827838,100.759434,100.759321
2022-10-27,100.128083,100.081857,99.944119,99.935764
2022-10-26,100.000000,100.000000,100.000000,100.000000
2022-10-25,99.325060,99.386722,99.532992,99.559047
...,...,...,...,...
2020-03-12,81.757577,81.354949,80.536452,81.063032
2020-03-11,87.324129,86.903117,85.855375,86.222664
2020-03-10,89.844994,89.400238,88.506781,88.846468
2020-03-09,88.054226,87.606194,86.941943,87.359443


In [199]:
class CumulativeOutperformanceCalculator(ReturnSeriesCalculator):

    def get_cumulative_return(self):
        self.cumulative_return_df = (
            self.return_comparison_df + 1)[::-1].cumprod(axis=0)[::-1]

    def get_cumulative_outperformance_df(self):
        pd.DataFrame({"SAA vs Benchmark": self.cumulative_return_df["SAA Returns"] - self.cumulative_return_df["Benchmark Returns"],
                      "Manager vs SAA": self.cumulative_return_df["Manager Returns"] - self.cumulative_return_df["SAA Returns"],
                      "Manager Drift vs Daily Rebalance": self.cumulative_return_df["Manager Returns With Drift"] - self.cumulative_return_df["Manager Returns"]})

In [200]:
test_co = CumulativeOutperformanceCalculator(benchmark_weights, saa_weights, manager_weights, returns)

In [201]:
test_co.get_all_returns()

In [202]:
test_co.get_cumulative_return()

In [209]:
test_co.cumulative_return_df

,Benchmark Returns,SAA Returns,Manager Returns,Manager Returns With Drift
2022-10-31,1.086802,1.091295,1.101121,1.096417
2022-10-28,1.088654,1.093017,1.105184,1.100504
2022-10-27,1.080920,1.084930,1.096241,1.091509
2022-10-26,1.079538,1.084043,1.096854,1.092211
2022-10-25,1.072251,1.077395,1.091731,1.087395
...,...,...,...,...
2020-03-12,0.882604,0.881922,0.883367,0.885379
2020-03-11,0.942697,0.942067,0.941708,0.941733
2020-03-10,0.969910,0.969137,0.970790,0.970391
2020-03-09,0.950578,0.949689,0.953626,0.954149


,SAA vs Benchmark,Manager vs SAA,Manager Drift vs Daily Rebalance
2022-10-31,0.004493,0.009826,-0.004704
2022-10-28,0.004363,0.012167,-0.004679
2022-10-27,0.004010,0.011311,-0.004732
2022-10-26,0.004505,0.012811,-0.004643
2022-10-25,0.005143,0.014337,-0.004337
...,...,...,...
2020-03-12,-0.000681,0.001445,0.002012
2020-03-11,-0.000630,-0.000359,0.000025
2020-03-10,-0.000774,0.001653,-0.000399
2020-03-09,-0.000890,0.003937,0.000523
